In [1]:
import pandas as pd

#df = pd.read_csv('train.csv')
#df_test = pd.read_csv('test.csv')

not_fit = True



In [2]:
"""
train_nlow = np.zeros((df['Page content'].values.shape[0], 4), dtype=object)
test_nlow = np.zeros((df_test['Page content'].values.shape[0], 4), dtype=object)

for i in range(df['Page content'].values.shape[0]):
    title, text, author, nf = procd.parse_html(df['Page content'][i], tolower=False)
    train_nlow[i, 0] = df['Id'][i]
    train_nlow[i, 1] = df['Popularity'][i]
    train_nlow[i, 2] = title
    train_nlow[i, 3] = text

for i in range(df_test['Page content'].values.shape[0]):
    title, text, author, nf = procd.parse_html(df_test['Page content'][i], tolower=False)
    test_nlow[i, 0] = df['Id'][i]
    test_nlow[i, 1] = df['Popularity'][i]
    test_nlow[i, 2] = title
    test_nlow[i, 3] = text

pd.DataFrame(data=train_nlow, columns=['Id', 'Popularity', 'title', 'content']).to_csv('train_parse_ncase.csv', index=False)
pd.DataFrame(data=test_nlow, columns=['Id', 'Popularity', 'title', 'content']).to_csv('test_parse_ncase.csv', index=False)
"""

"\ntrain_nlow = np.zeros((df['Page content'].values.shape[0], 4), dtype=object)\ntest_nlow = np.zeros((df_test['Page content'].values.shape[0], 4), dtype=object)\n\nfor i in range(df['Page content'].values.shape[0]):\n    title, text, author, nf = procd.parse_html(df['Page content'][i], tolower=False)\n    train_nlow[i, 0] = df['Id'][i]\n    train_nlow[i, 1] = df['Popularity'][i]\n    train_nlow[i, 2] = title\n    train_nlow[i, 3] = text\n\nfor i in range(df_test['Page content'].values.shape[0]):\n    title, text, author, nf = procd.parse_html(df_test['Page content'][i], tolower=False)\n    test_nlow[i, 0] = df['Id'][i]\n    test_nlow[i, 1] = df['Popularity'][i]\n    test_nlow[i, 2] = title\n    test_nlow[i, 3] = text\n\npd.DataFrame(data=train_nlow, columns=['Id', 'Popularity', 'title', 'content']).to_csv('train_parse_ncase.csv', index=False)\npd.DataFrame(data=test_nlow, columns=['Id', 'Popularity', 'title', 'content']).to_csv('test_parse_ncase.csv', index=False)\n"

In [3]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

[nltk_data] Downloading package stopwords to /home/cgv03/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import numpy as np
from bs4 import BeautifulSoup
from collections import defaultdict
import src.proc_data as procd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

"""
train_channel = np.zeros((df.values.shape[0], 4), dtype=object)
test_channel = np.zeros((df_test.values.shape[0], 4), dtype=object)
columns = ['entertainment', 'world', 'social-media', 'tech']


for i in range(df['Page content'].values.shape[0]):
    channel = procd.parse_channel(df['Page content'][i])
    if channel in columns:
        train_channel[i, columns.index(channel)] = 1

pd.DataFrame(data=train_channel, columns=columns).to_csv('train_channel.csv', index=False)
        
for i in range(df_test['Page content'].values.shape[0]):
    channel = procd.parse_channel(df_test['Page content'][i])
    if channel in columns:
        test_channel[i, columns.index(channel)] = 1
        
pd.DataFrame(data=test_channel, columns=columns).to_csv('test_channel.csv', index=False)
"""
"""
train_parse = np.zeros((df.values.shape[0], 5),dtype=object)

for i in range(df['Page content'].values.shape[0]):
#for i in range(5):

    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]\
            = procd.parse_html(df['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')

train_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)

#for i in range(5):

for i in range(df_test['Page content'].values.shape[0]):
    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3] \
            = procd.parse_html(df_test['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')
"""

"\ntrain_parse = np.zeros((df.values.shape[0], 5),dtype=object)\n\nfor i in range(df['Page content'].values.shape[0]):\n#for i in range(5):\n\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]            = procd.parse_html(df['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_b_parsed.csv')\n\ntrain_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)\n\n#for i in range(5):\n\nfor i in range(df_test['Page content'].values.shape[0]):\n    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]             = procd.parse_html(df_test['Page content'][i])\n    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))\n\npd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_b_parsed.csv')\n"

In [5]:
df = pd.read_csv('train_parsed.csv')
df_test = pd.read_csv('test_parsed.csv')
y_train = df['Popularity']

In [323]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
    
n_topics = 15
n_top_words = 15
n_features = 2**20
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                tokenizer=tokenizer_stem_nostop)
tf = tf_vectorizer.fit_transform(df['title'])

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=100,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
topic_tf = lda.fit_transform(tf)
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topic #0:
live phone bring song fire hack find see hous event camera white stream two imag
Topic #1:
new make world want instagram york launch kid citi break develop add drone share travel
Topic #2:
video star show top may thing week youtub buy war miss go help season dead
Topic #3:
game know need news moment dog throne offer good right yahoo realli creat cover captur
Topic #4:
one time life peopl reveal real super market busi amazon announc updat twitter xp put
Topic #5:
facebook tech home kill win us fight christma secur light heart million snowden epic pinterest
Topic #6:
googl twitter way best glass user cat offic made soni pay rais free street beauti
Topic #7:
ad take tv world work cup vine play ebola show test team time film challeng
Topic #8:
like social media look job trailer chang love ever open print save girl friend bad
Topic #9:
u come samsung turn smartphon obama big give galaxi data movi call brand nsa bill
Topic #10:
use internet ipad power plan car new robot ceo still s

In [281]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfdoc = TfidfVectorizer(max_features=2**20, tokenizer=tokenizer_stem_nostop)
"""
tfdoc.fit(df['content'])
doc = tfdoc.transform(df['content'])
"""

train_content = pd.read_csv('train_noun.csv')
tfdoc.fit(train_content['NN'])
doc = tfdoc.transform(train_content['NN'])

tfaut = TfidfVectorizer(max_features=2**10)
author = tfaut.fit_transform(df['author'])


In [282]:
from nltk.tree import Tree
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
"""
tag_types = ['FACILITY', 'GPE', 'GSP', 'LOCATION', 'ORGANIZATION', 'PERSON']

df_ncase = pd.read_csv('train_parse_ncase.csv')['content'].values
dt_ncase = pd.read_csv('test_parse_ncase.csv')['content'].values

train_ner = np.zeros((df_ncase.shape[0], 6 * 2), dtype=int)
test_ner  = np.zeros((dt_ncase.shape[0], 6 * 2), dtype=int)

for i in range(df_ncase.shape[0]):
    tokens = nltk.word_tokenize(df_ncase[i])
    tags = nltk.pos_tag(tokens)
    chunks = nltk.ne_chunk(tags)
    if i % 500 == 0:
        print('analyzing %d' %(i))
    for x in chunks:
        if isinstance(x, Tree):
            label = x.label()
            idx = tag_types.index(label)
            train_ner[i, idx] = 1
            train_ner[i, idx+6] += 1

pd.DataFrame(data=train_ner, columns=['has_FACILITY', 'has_GPE', 'has_GSP', 'has_LOCATION', 
                                      'has_ORGANIZATION', 'has_PERSON','FACILITY', 'GPE', 
                                      'GSP', 'LOCATION', 'ORGANIZATION', 'PERSON']).to_csv('train_ner.csv',index=False)
            
for i in range(dt_ncase.shape[0]):
    tokens = nltk.word_tokenize(dt_ncase[i])
    tags = nltk.pos_tag(tokens)
    chunks = nltk.ne_chunk(tags)
    for x in chunks:
        if isinstance(x, Tree):
            label = x.label()
            idx = tag_types.index(label)
            test_ner[i, idx] = 1
            test_ner[i, idx+6] += 1
pd.DataFrame(data=test_ner, columns=['has_FACILITY', 'has_GPE', 'has_GSP', 'has_LOCATION', 
                                      'has_ORGANIZATION', 'has_PERSON','FACILITY', 'GPE', 
                                      'GSP', 'LOCATION', 'ORGANIZATION', 'PERSON']).to_csv('test_ner.csv',index=False)
"""

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/cgv03/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/cgv03/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/cgv03/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/cgv03/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"\ntag_types = ['FACILITY', 'GPE', 'GSP', 'LOCATION', 'ORGANIZATION', 'PERSON']\n\ndf_ncase = pd.read_csv('train_parse_ncase.csv')['content'].values\ndt_ncase = pd.read_csv('test_parse_ncase.csv')['content'].values\n\ntrain_ner = np.zeros((df_ncase.shape[0], 6 * 2), dtype=int)\ntest_ner  = np.zeros((dt_ncase.shape[0], 6 * 2), dtype=int)\n\nfor i in range(df_ncase.shape[0]):\n    tokens = nltk.word_tokenize(df_ncase[i])\n    tags = nltk.pos_tag(tokens)\n    chunks = nltk.ne_chunk(tags)\n    if i % 500 == 0:\n        print('analyzing %d' %(i))\n    for x in chunks:\n        if isinstance(x, Tree):\n            label = x.label()\n            idx = tag_types.index(label)\n            train_ner[i, idx] = 1\n            train_ner[i, idx+6] += 1\n\npd.DataFrame(data=train_ner, columns=['has_FACILITY', 'has_GPE', 'has_GSP', 'has_LOCATION', \n                                      'has_ORGANIZATION', 'has_PERSON','FACILITY', 'GPE', \n                                      'GSP', 'LOCATION', 'ORGA

In [373]:
from sklearn.feature_selection import SelectKBest, chi2

ch2 = SelectKBest(chi2, k=200)
doc_x = ch2.fit_transform(doc, y_train)

cha2 = SelectKBest(chi2, k=10)
aut_x = cha2.fit_transform(author, y_train)


In [359]:
pkl.dump(lda, open('lda_title15', 'wb'))

In [361]:
import pickle as pkl

lda_title = pkl.load(open('lda_title15', 'rb'))
lda_content = pkl.load(open('lda_content.pkl', 'rb'))

/home/cgv03/anaconda3/envs/ml_mid_env/lib/python3.5/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.18.1 when using version 0.18. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [362]:
train_content = pd.read_csv('train_noun.csv')

tfc_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                tokenizer=tokenizer_stem_nostop)
tfc = tfc_vectorizer.fit_transform(df['content'])

tft_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                tokenizer=tokenizer_stem_nostop)
tft = tft_vectorizer.fit_transform(df['title'])


In [363]:
topic_tc = lda_content.transform(tfc)
topic_tt = lda_title.transform(tft)

In [285]:
n_twords = np.zeros((df['title'].shape[0], 1), dtype=int)
n_twords_test = np.zeros((df_test['title'].shape[0], 1), dtype=int)
for i in range(n_twords.shape[0]):
    n_twords[i] = len(tokenizer_stem_nostop(df['title'][i]))
for i in range(n_twords_test.shape[0]):
    n_twords_test[i] = len(tokenizer_stem_nostop(df_test['title'][i]))

In [286]:
def mix(a, b):
    r = a.shape[1]
    k = b.shape[1]
    temp = None
    for i in range(r):
        for j in range(k):
            if temp: temp = a[:, i].reshape(-1,1) * b[:,j].reshape(-1,1)
            else: temp = np.concatenate((temp, a[:, i].reshape(-1,1) * b[:,j].reshape(-1,1)), axis=1)
    
    return temp

In [374]:
from scipy.sparse import hstack
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
stdchi = SelectKBest(chi2, k=150)
poly_stdn = PolynomialFeatures(degree=2)
poly_cont = PolynomialFeatures(degree=2)
poly_mixd = PolynomialFeatures(degree=2)
train_weekday = pd.read_csv('train_weekday.csv').values
train_time = pd.read_csv('train_time.csv').values
train_channel = pd.read_csv('train_channel_34.csv').values
train_polarity = pd.read_csv('train_polarity.csv').values
train_t_polarity = pd.read_csv('train_t_polarity.csv').values
train_links = pd.read_csv('train_links.csv').values
train_keypop = pd.read_csv('train_keypop.csv').values
train_ner = pd.read_csv('train_ner.csv').values
train_hnum = pd.read_csv('train_hnum.csv').values
train_extra = pd.read_csv('train_extra.csv').values
train_extra2 = pd.read_csv('train_extra2.csv').values
train_tweet = pd.read_csv('train_tweet.csv').values
train_tweet[train_tweet>0] = 1
#train_ner = train_ner[:, 6:]
train_nwords = df['n_words'].values.reshape(-1,1)
train_nfig = df['n_fig'].values.reshape(-1,1)

time_feats = np.concatenate((train_time, train_weekday, train_channel), axis=1)
cont_feats = np.concatenate((train_polarity, train_t_polarity,  topic_tt), axis=1)
stdn_feats = np.concatenate((n_twords, train_nwords, train_extra), axis=1)
mixd_feats = np.concatenate((train_time, train_polarity), axis=1)
stdn_feats = sc.fit_transform(stdn_feats)
#stdn_feats = poly_stdn.fit_transform(stdn_feats)[:,1:]
#cont_feats = poly_cont.fit_transform(cont_feats)[:,1:]
"""
ohot_feats = np.concatenate((train_weekday, train_channel, topic_tf), axis=1)
stdn_feats = np.concatenate((n_twords, train_nwords, train_ner[:,6:], train_polarity, train_time), axis=1)
stdn_feats = sc.fit_transform(stdn_feats)
stdn_feats = poly_stdn.fit_transform(stdn_feats)[:,1:]
"""

"""
ohot_feats = np.concatenate((train_weekday, train_channel), axis=1)
stdn_feats = np.concatenate((train_time, train_polarity, topic_tf, n_twords, train_nwords, train_ner[:,6:]), axis=1)
mixd_feats = np.concatenate((train_channel, train_polarity, train_time, train_ner[:,6:]), axis=1)
stdn_feats = sc.fit_transform(stdn_feats)
stdn_feats = poly_stdn.fit_transform(stdn_feats)[:,1:]
mixd_feats = poly_mixd.fit_transform(mixd_feats)[:,1:]
"""

X_train_aug = np.concatenate((time_feats, cont_feats, stdn_feats,doc_x.todense()), axis=1)
#X_train_aug = np.concatenate((ohot_feats, stdn_feats, doc_x.todense()), axis=1)
#X_train_aug = stdchi.fit_transform(np.fabs(X_train_aug), y_train)

#X_train_aug = sc.fit_transform(X_train_aug)

In [376]:
from sklearn.model_selection import train_test_split
# hold out testing set
y_train = df['Popularity']
X_htrain, X_htest, y_htrain, y_htest = train_test_split(X_train_aug, y_train, test_size=0.3, random_state=0)

print(X_train_aug.shape)

(27643, 270)


In [366]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [378]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
#param_c = { 'C': [0.001, 0.005, 0.01, 0.03, 0.05, 0.07, 0.1, 0.25, 0.5, 0.7, 1.0, 3.0, 10.0, 100.0, 200.0, 1000, 1000.0]}
#grid_search = GridSearchCV(LogisticRegression(random_state=0), param_grid=param_c)
#start = time()
#grid_search.fit(X_train_aug, y_train)

#print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
#      % (time() - start, len(grid_search.cv_results_['params'])))
#report(grid_search.cv_results_)
candidates = []



# former best 0.06
# 700,21
lr = RandomForestClassifier(n_estimators=700, max_depth=21, criterion='entropy', random_state=0, n_jobs=8)
#lr = SGDClassifier(loss='log', penalty='elasticnet', alpha=0.000009, l1_ratio=0.35, 
#                   fit_intercept=True, n_iter=3000, shuffle=True, verbose=0, epsilon=0.1, 
#                   n_jobs=8, random_state=0)
score = cross_val_score(lr, X_train_aug, y_train, cv=5, n_jobs=8)

lr.fit(X_train_aug, y_train)
pred = lr.predict(X_train_aug)
pred_t = lr.predict(X_htest)
score_train = metrics.accuracy_score(y_train, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

confmat = confusion_matrix(y_true=y_train, y_pred=pred)
fpr, tpr, thresholds = metrics.roc_curve(y_train, pred, pos_label=1)
com = auc(fpr, tpr)
print("LR score (train):%f , (test):%f , (CV):%f " %(score_train, score_test, score.mean()))
candidates.append({'clf':lr, 'C':0.5, 'score':score.mean})
print(confmat)
print(com)
"""
LR score (train):0.880295 , (test):0.881707 , (CV):0.572912 
[[13749   262]
 [ 3047 10585]]
0.878891107167


"""

LR score (train):0.888652 , (test):0.892801 , (CV):0.574142 
[[13759   252]
 [ 2826 10806]]
0.887353896863


'\nLR score (train):0.880295 , (test):0.881707 , (CV):0.572912 \n[[13749   262]\n [ 3047 10585]]\n0.878891107167\n\n\n'

In [259]:
from sklearn.ensemble import VotingClassifier

candidates = []

rf1 = RandomForestClassifier(n_estimators=300, max_depth=12, criterion='entropy', random_state=0)
rf2 = RandomForestClassifier(n_estimators=400, max_depth=14, criterion='entropy', random_state=0)
rf3 = RandomForestClassifier(n_estimators=500, max_depth=19, criterion='entropy', random_state=0)
rf4 = RandomForestClassifier(n_estimators=600, max_depth=21, criterion='entropy', random_state=0)
rf5 = RandomForestClassifier(n_estimators=700, max_depth=21, criterion='entropy', random_state=0)

vc = VotingClassifier(estimators=[('rf1', rf1), ('rf2', rf2), ('rf3', rf3), ('rf4', rf4), ('rf5', rf5)], voting='soft', weights=None, n_jobs=1)

score = cross_val_score(vc, X_train_aug, y_train, cv=5, n_jobs=8)

vc.fit(X_train_aug, y_train)

pred = vc.predict(X_train_aug)
pred_t = vc.predict(X_htest)
score_train = metrics.accuracy_score(y_train, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

print("VC score (train):%f , (test):%f , (CV):%f " %(score_train, score_test, score.mean()))
candidates.append({'clf':vc, 'C':0.5, 'score':score.mean})

/home/cgv03/anaconda3/envs/ml_mid_env/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/cgv03/anaconda3/envs/ml_mid_env/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/cgv03/anaconda3/envs/ml_mid_env/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/cgv03/anaconda3/envs/ml_mid_env/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/cgv03/anaconda3/envs/ml_mid_env/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multipr

VC score (train):0.822451 , (test):0.824913 , (CV):0.570343 


In [251]:
candidates = []

lr = LogisticRegression(C=1., random_state=0)

score = cross_val_score(lr, X_train_aug, y_train, cv=5)

lr.fit(X_train_aug, y_train)
pred = lr.predict(X_train_aug)
pred_t = lr.predict(X_htest)
score_train = metrics.accuracy_score(y_train, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

print("LR score (train):%f , (test):%f , (CV):%f " %(score_train, score_test, score.mean()))
candidates.append({'clf':lr, 'C':0.5, 'score':score.mean})

LR score (train):0.570452 , (test):0.573857 , (CV):0.562204 


In [369]:
topic_tc_test = tfc_vectorizer.transform(df_test['content'])
topic_tc_test = lda_content.transform(topic_tc_test)

topic_tt_test = tft_vectorizer.transform(df_test['title'])
topic_tt_test = lda_title.transform(topic_tt_test)

In [380]:
'''
doc_test = tfdoc.transform(df_test['content'])
test_doc_x = ch2.transform(doc_test)
'''
#test_content = pd.read_csv('test_noun.csv')
#doc_test = tfdoc.transform(test_content['NN'])
test_doc_x = ch2.transform(doc_test)



#aut_test = tfaut.transform(df_test['author'])
#test_aut_x = cha2.transform(aut_test)

In [381]:
test_weekday = pd.read_csv('test_weekday.csv').values
test_time = pd.read_csv('test_time.csv').values
test_channel = pd.read_csv('test_channel_34.csv').values
test_polarity = pd.read_csv('test_polarity.csv').values
test_t_polarity = pd.read_csv('test_t_polarity.csv').values

test_links = pd.read_csv('test_links.csv').values
test_keypop = pd.read_csv('test_keypop.csv').values
test_nwords = df_test['n_words'].values.reshape(-1,1)
test_nfig = df_test['n_fig'].values.reshape(-1,1)
test_nword = df_test['n_words'].reshape(-1,1)
test_ner = pd.read_csv('test_title_ner.csv').values
test_hnum = pd.read_csv('test_hnum.csv').values
test_extra = pd.read_csv('test_extra.csv').values
test_tweet = pd.read_csv('test_tweet.csv').values
test_tweet[test_tweet > 0] = 1
time_feats = np.concatenate((test_time, test_weekday, test_channel), axis=1)
cont_feats = np.concatenate((test_polarity, test_t_polarity, topic_tt_test), axis=1)
stdn_feats = np.concatenate((n_twords_test,test_nwords, test_extra), axis=1)
mixd_feats = np.concatenate((test_time, test_polarity), axis=1)
stdn_feats = sc.transform(stdn_feats)

#cont_feats = poly_cont.transform(cont_feats)[:,1:]
#stdn_feats = poly_stdn.transform(stdn_feats)[:,1:]

X_test_con = np.concatenate((time_feats, cont_feats, stdn_feats, test_doc_x.todense()), axis=1)

"""
ohot_feats = np.concatenate((test_weekday, test_channel, topic_test), axis=1)
stdn_feats = np.concatenate((n_twords_test, test_nwords, test_ner[:,6:], test_polarity, test_time), axis=1)
stdn_feats = sc.fit_transform(stdn_feats)
#stdn_feats = stdchi.transform(np.fabs(stdn_feats))
stdn_feats = poly_stdn.transform(stdn_feats)[:,1:]

X_test_con = np.concatenate((ohot_feats, stdn_feats, test_doc_x.todense()), axis=1)
#X_test_con = sc.transform(X_test_con)
"""

/home/cgv03/anaconda3/envs/ml_mid_env/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


'\nohot_feats = np.concatenate((test_weekday, test_channel, topic_test), axis=1)\nstdn_feats = np.concatenate((n_twords_test, test_nwords, test_ner[:,6:], test_polarity, test_time), axis=1)\nstdn_feats = sc.fit_transform(stdn_feats)\n#stdn_feats = stdchi.transform(np.fabs(stdn_feats))\nstdn_feats = poly_stdn.transform(stdn_feats)[:,1:]\n\nX_test_con = np.concatenate((ohot_feats, stdn_feats, test_doc_x.todense()), axis=1)\n#X_test_con = sc.transform(X_test_con)\n'

In [382]:

for cand in candidates:
    predict = cand['clf'].predict(X_test_con)
    output = np.zeros((X_test_con.shape[0], 2), dtype=int)
    output[:, 0] = df_test['Id']
    output[:, 1] = predict
    df_output = pd.DataFrame(data=output, columns=['Id', 'Popularity'])
    df_output.to_csv('test___%f.csv' % (cand['C']), index=False)